Preparação dos arquivos

In [2]:
from modules.utilities import prepare_data_filesystem

prepare_data_filesystem()

ERROR:root:O diretório data/landing já existe, nada será feito.
ERROR:root:O diretório data/bronze já existe, nada será feito.
ERROR:root:O diretório data/silver já existe, nada será feito.
ERROR:root:O diretório data/gold já existe, nada será feito.


Etapa Landing (Extraindo a lista de IDs)

In [2]:
from modules.etl import Landing

Landing.full_process(database='nucleotide',term='Chikungunya',retmax=1000,datetype="pdat",reldate=60)

Etapa bronze (Extraindo as informações dos IDs do XML salvo pela etapa Landing)

In [3]:
from modules.etl import Bronze

Bronze.full_process('nucleotide',rettype='gbc',retmode='xml')

Etapa Silver (Das informações dos IDs retirar o que é interessante e jogar em um formato de arquivo mais adequado para manipulação) -> Em testes ainda

In [34]:
import os
import xmltodict
import pandas as pd
from datetime import datetime

bronze_path = 'data/bronze'
lista_de_xmls = [os.path.join(bronze_path,file) for file in os.listdir(bronze_path)]

full_df = pd.DataFrame()
for xml in lista_de_xmls:
    with open(xml,'r') as xml_file:
        xml_data = xml_file.read()
    full_df = pd.concat([full_df,pd.DataFrame(xmltodict.parse(xml_data).get('INSDSet').get('INSDSeq'))],ignore_index=True)

In [35]:
# Renomear coluna 'INSDSeq_locus' para 'locus'
colunas_a_serem_renomeadas = {
    'INSDSeq_locus': 'locus',
    'INSDSeq_length':'length',
    'INSDSeq_update-date': 'update_date',
    'INSDSeq_create-date': 'create_date'
    }
full_df = full_df.rename(columns=colunas_a_serem_renomeadas) 

# Alterar tipo das colunas
colunas_a_serem_convertidas = {
    'length': 'int64',
    'update_date':'datetime64[ns]',
    'create_date':'datetime64[ns]'
    }
full_df = full_df.astype(colunas_a_serem_convertidas)

# Remover colunas: 'INSDSeq_strandedness', 'INSDSeq_moltype' e 10 outras colunas
colunas_a_serem_removidas = ['INSDSeq_strandedness',
                             'INSDSeq_moltype',
                             'INSDSeq_topology',
                             'INSDSeq_division',
                             'INSDSeq_definition',
                             'INSDSeq_primary-accession',
                             'INSDSeq_accession-version',
                             'INSDSeq_other-seqids',
                             'INSDSeq_source',
                             'INSDSeq_organism',
                             'INSDSeq_taxonomy']
full_df = full_df.drop(columns=colunas_a_serem_removidas)

In [49]:
count = 1
for cell in full_df['INSDSeq_references']:
    print(f"sequencia {count}")
    if not isinstance(cell['INSDReference'], list):
        cell['INSDReference'] = [cell.get("INSDReference")]

    for item in cell['INSDReference']:
        print("Pubmed")
        #print(item.get('INSDReference_xref',None).get('INSDReference_pubmed',None))
        print(item)
    count+=1

sequencia 1
Pubmed
{'INSDReference_reference': '1', 'INSDReference_position': '1..11655', 'INSDReference_authors': {'INSDAuthor': ['Dangsagul,W.', 'Puthavathana,P.', 'Rojanawiwat,A.', 'Meephaendee,L.', 'Posanacharoen,A.', 'Sereewit,J.', 'Sobolik,E.B.', 'Roychoudhury,P.', 'Greninger,A.L.']}, 'INSDReference_title': 'Direct Submission', 'INSDReference_journal': 'Submitted (04-APR-2024) Virology, University of Washington, 850 Republican St, Seattle, WA 98109, USA'}
sequencia 2
Pubmed
{'INSDReference_reference': '1', 'INSDReference_position': '1..11483', 'INSDReference_authors': {'INSDAuthor': ['Dangsagul,W.', 'Puthavathana,P.', 'Rojanawiwat,A.', 'Meephaendee,L.', 'Posanacharoen,A.', 'Sereewit,J.', 'Sobolik,E.B.', 'Roychoudhury,P.', 'Greninger,A.L.']}, 'INSDReference_title': 'Direct Submission', 'INSDReference_journal': 'Submitted (04-APR-2024) Virology, University of Washington, 850 Republican St, Seattle, WA 98109, USA'}
sequencia 3
Pubmed
{'INSDReference_reference': '1', 'INSDReference_p